we're scraping to grab job title, date of posting, company, required skills for the job job description

To know the most in demand job we want to have the jobs, with their date of posting. so we grab thsoe posted over  years then we simply check the amount jobs that lead i.e the most jobs demanded for within the last 5 years

for the recommending we want to train the model on the skills as features and the jobs as a target that is required skills needed for a job determins the type of job to e recommended so a user can just input his skills and jobs are recommended for him based on that

#### WEB SCRAPING

In [1]:
# first, the required imports
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import requests


In [2]:
def get_url(position):
    """Generates a url for each position(web developer, etc)"""
    url_template = 'https://ng.indeed.com/jobs?q={}&start=0'
    url = url_template.format(position)
    return url

In [3]:
url = get_url('Web developer')
url

'https://ng.indeed.com/jobs?q=Web developer&start=0'

In [4]:
# Extracting the raw html
# Access the url with the request object
response = requests.get(url)
response

<Response [200]>

In [5]:
# Scrape the web page content
web_page = response.text
# Create a soup object(sl_soup) for parsing the webpage using html parser
sl_soup = BeautifulSoup(web_page, 'html.parser')

In [6]:
cards = sl_soup.findAll('div', 'cardOutline')
len(cards)

15

In [19]:
# Prototype the extraction of a single record/card
card = cards[1]

In [32]:
atag = card.h2.a
job_title = atag.span.get('title')
job_title

'Frontend Developer'

In [33]:
job_url = 'https://ng.indeed.com' + atag.get('href')
job_url

'https://ng.indeed.com/company/Sabi/jobs/Frontend-Developer-b2eb96beda787da6?fccid=6a333fef94a2ad5e&vjs=3'

In [22]:
company_name = card.find('span', 'companyName').text
company_name

'Sabi'

In [23]:
company_location = card.find('div', {'class': 'companyLocation'}).text
company_location

'Lagos'

In [24]:
job_description = card.find('div', 'job-snippet').text.strip()
job_description

'Craft code that meets our internal standards for style, maintainability, and best practices for a high-scale web environment.'

In [36]:
date_posted = card.find('span', 'date').text[6:]
date_posted

'Posted 1 day ago'

In [26]:
today = datetime.today().strftime('%Y-%m-%d')
today

'2022-07-30'

In [27]:
try:
    job_salary = card.find('div', 'metadata salary-snippet-container').text
except:
    job_salary = ''

#     Alternatively with if block
# if card.find('div', 'metadata salary-snippet-container') == None:
#     salary = None
# else:
#     salary = card.find('div', 'metadata salary-snippet-container').text
job_salary

''

#Generalize the model with a function and getting results form a single page.

In [40]:
def get_job_record(card):
    """Extracts job data from a single card"""
    atag = card.h2.a
    job_title = atag.span.get('title')
    job_url = 'https://ng.indeed.com' + atag.get('href')
    company_name = card.find('span', 'companyName').text
    company_location = card.find('div', {'class': 'companyLocation'}).text
    job_description = card.find('div', 'job-snippet').text.strip()
    date_posted = card.find('span', 'date').text[6:]
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('div', 'metadata salary-snippet-container').text
    except:
        job_salary = ''
    record = (job_title, company_name, company_location, job_description, job_salary, date_posted, today, job_url,)
    return record

In [41]:
records = []

for card in cards:
    record = get_job_record(card)
    records.append(record)

In [42]:
records[:2]

[('FRONT END DEVELOPER',
  'ICE COMMERCIAL POWER',
  'Lagos',
  'Front end developers are responsible for ensuring the alignment of web design and user experience requirements, optimizing web pages for maximum efficiency, and…',
  '₦200,000 a month',
  'Posted 3 days ago',
  '2022-07-30',
  'https://ng.indeed.com/company/ICE-COMMERCIAL-POWER/jobs/Front-End-Developer-bcd37ee321ecd681?fccid=db5dd3a48943e57e&vjs=3'),
 ('Frontend Developer',
  'Sabi',
  'Lagos',
  'Craft code that meets our internal standards for style, maintainability, and best practices for a high-scale web environment.',
  '',
  'Posted 1 day ago',
  '2022-07-30',
  'https://ng.indeed.com/company/Sabi/jobs/Frontend-Developer-b2eb96beda787da6?fccid=6a333fef94a2ad5e&vjs=3')]

#Getting the next page

In [47]:
while True:
    try:
        page_url = 'https://ng.indeed.com' + sl_soup.find('a', {'aria-label' : 'Next'}).get('href')
    except AttributeError:
        break
    
    response = requests.get(url)
    sl_soup = BeautifulSoup(response.text, 'html.parser')
    cards = sl_soup.findAll('div', 'cardOutline')
    for card in cards:
        record = get_job_record(card)
        records.append(record)
    

390